In [2]:
from pprint import pprint
from sklearn.datasets import fetch_20newsgroups
newsgroups_train=fetch_20newsgroups(subset="train",shuffle=True)
newsgroups_test=fetch_20newsgroups(subset="test",shuffle=True)
#pprint(newsgroups_train)
#pprint(newsgroups_train.target_names)

In [3]:
pprint(newsgroups_train.filenames)

array([ 'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.autos\\102994',
       'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51861',
       'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.mac.hardware\\51879',
       ...,
       'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60695',
       'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.graphics\\38319',
       'C:\\Users\\hp\\scikit_learn_data\\20news_home\\20news-bydate-train\\rec.motorcycles\\104440'],
      dtype='<U92')


In [4]:
pprint(newsgroups_train.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])


In [5]:
print("\n".join(newsgroups_train.data[0].split("\n")[:3]))#prints first line of the first data file

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(newsgroups_train.data)#it returns a Document-Term matrix. [n_samples, n_features].
X_train_counts.shape

(11314, 130107)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer# removes common words like (the, is, an) 
tfidf_transformer=TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, newsgroups_train.target)


In [9]:
from sklearn.pipeline import Pipeline
text_clf=Pipeline([('vect',CountVectorizer()),
                  ('tfidf',TfidfTransformer()),
                  ('clf',MultinomialNB()),])
text_clf=text_clf.fit(newsgroups_train.data,newsgroups_train.target)

In [10]:
import numpy as np
#newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(newsgroups_test.data)
np.mean(predicted == newsgroups_test.target)

0.7738980350504514

In [11]:
from nltk.corpus import stopwords

In [12]:
from sklearn.pipeline import Pipeline
text_clf=Pipeline([('vect',CountVectorizer(stop_words='english')),#Removing stop words
                  ('tfidf',TfidfTransformer()),
                  ('clf',MultinomialNB(fit_prior=False)),])
text_clf=text_clf.fit(newsgroups_train.data,newsgroups_train.target)

In [13]:
import numpy as np
#newsgroups_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(newsgroups_test.data)
np.mean(predicted == newsgroups_test.target)

0.8214285714285714

In [14]:
import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer   #stemming reduces derived words like fishing,fisher to fish
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer,self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                             ('tfidf', TfidfTransformer()),
                             ('mnb', MultinomialNB(fit_prior=False)),])
text_mnb_stemmed = text_mnb_stemmed.fit(newsgroups_train.data, newsgroups_train.target)
predicted_mnb_stemmed = text_mnb_stemmed.predict(newsgroups_test.data)
np.mean(predicted_mnb_stemmed == newsgroups_test.target)

0.81678173127987252

In [1]:
#pprint(newsgroups_test)